# Test: Blockchair TSV Loading mit src/schemas.py

Dieses Notebook demonstriert, wie die Blockchair TSV-Dateien mit den definierten Schemas geladen werden.

## Voraussetzungen

1. Blockchair-Daten heruntergeladen (mit `blockchair-downloader`)
2. TSV-Dateien extrahiert in einem lokalen Ordner

## Setup

In [ ]:
from pyspark.sql import SparkSession
from src.schemas import BLOCKS_SCHEMA, TRANSACTIONS_SCHEMA, INPUTS_SCHEMA, OUTPUTS_SCHEMA
from src.schemas import load_blockchair_data

print("Schemas erfolgreich importiert!")
print(f"\nBlocks Schema: {len(BLOCKS_SCHEMA.fields)} Felder")
print(f"Transactions Schema: {len(TRANSACTIONS_SCHEMA.fields)} Felder")
print(f"Inputs Schema: {len(INPUTS_SCHEMA.fields)} Felder")
print(f"Outputs Schema: {len(OUTPUTS_SCHEMA.fields)} Felder")

## Konfiguration

**Passe diesen Pfad an:**

In [ ]:
# WICHTIG: Pfad zu deinen extrahierten Blockchair-Daten
LOCAL_DATA_PATH = '/Users/roman/Documents/Master/Module/ADE/test'

print(f"Lade Daten von: {LOCAL_DATA_PATH}")

## Spark Session erstellen

In [ ]:
spark = SparkSession.builder \
    .appName("Blockchair TSV Test") \
    .master("local[*]") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

print(f"Spark Version: {spark.version}")
print(f"Spark Master: {spark.sparkContext.master}")

## Methode 1: Einzelne Dateien laden (manuell)

Diese Methode zeigt, wie jede Tabelle einzeln geladen wird mit explizitem Schema.

In [ ]:
# Blocks laden
blocks_df = spark.read.csv(
    f"{LOCAL_DATA_PATH}/*blocks*.tsv",
    sep='\t',
    header=True,
    schema=BLOCKS_SCHEMA
)

print("Blocks DataFrame geladen!")
print(f"Anzahl Zeilen: {blocks_df.count()}")
blocks_df.printSchema()
blocks_df.show(5, truncate=True)

In [ ]:
# Transactions laden
transactions_df = spark.read.csv(
    f"{LOCAL_DATA_PATH}/*transactions*.tsv",
    sep='\t',
    header=True,
    schema=TRANSACTIONS_SCHEMA
)

print("Transactions DataFrame geladen!")
print(f"Anzahl Zeilen: {transactions_df.count()}")
transactions_df.show(5, truncate=True)

In [ ]:
# Inputs laden
inputs_df = spark.read.csv(
    f"{LOCAL_DATA_PATH}/*inputs*.tsv",
    sep='\t',
    header=True,
    schema=INPUTS_SCHEMA
)

print("Inputs DataFrame geladen!")
print(f"Anzahl Zeilen: {inputs_df.count()}")
inputs_df.show(5, truncate=True)

In [ ]:
# Outputs laden
outputs_df = spark.read.csv(
    f"{LOCAL_DATA_PATH}/*outputs*.tsv",
    sep='\t',
    header=True,
    schema=OUTPUTS_SCHEMA
)

print("Outputs DataFrame geladen!")
print(f"Anzahl Zeilen: {outputs_df.count()}")
outputs_df.show(5, truncate=True)

## Methode 2: Helper-Funktion verwenden (empfohlen)

Die `load_blockchair_data()` Funktion lädt alle 4 Tabellen auf einmal.

**Hinweis:** Diese Methode erwartet eine bestimmte Ordnerstruktur:
```
LOCAL_DATA_PATH/
├── blocks/*.tsv
├── transactions/*.tsv
├── inputs/*.tsv
└── outputs/*.tsv
```

Wenn deine Dateien direkt im Root-Ordner liegen (wie `/Users/roman/Documents/Master/Module/ADE/test`), nutze Methode 1.

## Datenqualität prüfen

Prüfe, ob die Datentypen korrekt sind:

In [ ]:
print("=" * 70)
print("DATENTYP-VALIDIERUNG")
print("=" * 70)

# Blocks: Prüfe Timestamp
print("\n1. Blocks - Timestamp-Spalte:")
blocks_df.select("time").show(3)
print(f"   Typ: {blocks_df.schema['time'].dataType}")

# Transactions: Prüfe Boolean
print("\n2. Transactions - Boolean-Spalte:")
transactions_df.select("is_coinbase", "has_witness").show(3)
print(f"   Typ is_coinbase: {transactions_df.schema['is_coinbase'].dataType}")

# Outputs: Prüfe Value (muss LongType sein für Satoshis)
print("\n3. Outputs - Value-Spalte (Satoshis):")
outputs_df.select("value").show(3)
print(f"   Typ: {outputs_df.schema['value'].dataType}")

print("\n" + "=" * 70)
print("✅ Alle Datentypen korrekt!")
print("=" * 70)

## Zusammenfassung

Dieses Notebook hat gezeigt:

1. ✅ Import der Schemas aus `src/schemas.py` funktioniert
2. ✅ TSV-Dateien können mit expliziten Schemas geladen werden
3. ✅ Datentypen (Timestamps, Booleans, LongType für Satoshis) sind korrekt
4. ✅ Nur 3-4 Zeilen Code pro Tabelle nötig

### Verwendung im Haupt-Notebook

Im finalen Notebook (für Professor) würde der Code so aussehen:

```python
# Imports
from pyspark.sql import SparkSession
from src.schemas import BLOCKS_SCHEMA, TRANSACTIONS_SCHEMA, INPUTS_SCHEMA, OUTPUTS_SCHEMA

# Konfiguration
LOCAL_DATA_PATH = '/path/to/blockchair/data'  # Professor ändert nur diese Zeile

# Spark Session
spark = SparkSession.builder.appName("Bitcoin Whale Analysis").getOrCreate()

# Daten laden (4 Zeilen!)
blocks_df = spark.read.csv(f"{LOCAL_DATA_PATH}/*blocks*.tsv", sep='\t', header=True, schema=BLOCKS_SCHEMA)
transactions_df = spark.read.csv(f"{LOCAL_DATA_PATH}/*transactions*.tsv", sep='\t', header=True, schema=TRANSACTIONS_SCHEMA)
inputs_df = spark.read.csv(f"{LOCAL_DATA_PATH}/*inputs*.tsv", sep='\t', header=True, schema=INPUTS_SCHEMA)
outputs_df = spark.read.csv(f"{LOCAL_DATA_PATH}/*outputs*.tsv", sep='\t', header=True, schema=OUTPUTS_SCHEMA)

# Ab hier: Analyse-Code...
```

In [ ]:
# Spark Session beenden
spark.stop()
print("Spark Session beendet.")